# Sparse broadcasting benchmarks

In [8]:
import scipy.sparse
import numpy as np

In [2]:
import scipy.sparse

In [176]:
from sklearn.utils.fixes import _astype_copy_false

def _multiply_sparse_1d(X, y, format_: str ='csc', copy: bool=True):
    """Multiply a scipy.sparse matrix by a 1d array row or column wise
    
    Parameters
    ----------
    X : scipy.sparse
    y : array
    format : str
    """
    X = X.asformat(format_, **_astype_copy_false(X))
    if (X.shape[0] != y.shape[0]) and (X.shape[1] != y.shape[0]):
        raise ValueError(
            f"Can't broadcast array:"
            f"X.shape={X.shape} and y.shape={y.shape}"
        )
    if copy:
        X = X.copy()
    X.data *= y[X.indices] 
    return X


def _has_shape_one(x):
    return x.shape[0] == 1 or x.shape[1] == 1


def safe_multiply_broadcast(X, Y, copy=True):
    """Multiply 2D array, with broadcasting and scipy.sparse support
       
    Without broadcasting support in scipy.sparse, the current workaround
    is to convert 1D array a 2D diagonal
    sparse array and perform a dot product which is significantly slower.
    
    Parameters
    ----------
    X : {scipy.sparse.csr_matrix, np.ndarray}, shape=(n_x0, n_x1)
       a csr_matrix or a 1D array
    y : {scipy.sparse.csr_matrix, np.ndarray}, shape=(n_y0, n_x1)
       a csr_matrix or a 1D array
      
    Returns
    -------
    >>> import numpy as np
    >>> import scipy.sparse
    >>> 
    """
    if not (X.ndim == 2 and Y.ndim == 2):
        raise ValueError(
            f'Input array/matrices must be 2D, '
            f'got X.shape={X.shape}, Y.shape={Y.shape}'
        )   
    
    if scipy.sparse.issparse(X) and isinstance(Y, np.ndarray) and _has_shape_one(Y):
        if Y.shape[0] == 1:
            return _multiply_sparse_1d(X, Y[0], format_='csr', copy=copy)
        elif Y.shape[1] == 1:
            return _multiply_sparse_1d(X, Y[:, 0], format_='csc', copy=copy)
        else:
            raise ValueError(f'Could not multiply X={X} and Y={Y}')
    elif scipy.sparse.issparse(Y) and isinstance(X, np.ndarray) and _has_shape_one(X):
        # multiplication is commutative
        return safe_multiply_broadcast(Y, X, copy=copy)  
    else:
        if copy is False:
            raise NotImplementedError
        return np.multiply(X, Y)

In [1]:
!pip install memory_profiler

In [179]:
n = 10000
m = 20000

X = scipy.sparse.rand(n, m, density=0.001, random_state=1, format='csr')

In [180]:
w = np.arange(m) + 1
W_diag = scipy.sparse.diags(w, shape=(m, m), offsets=0, format='csr')

In [181]:
%%timeit 

X * W_diag

2.09 ms ± 33.6 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [182]:
%%timeit

X2 = safe_multiply_broadcast(X, w[None, :])

477 µs ± 6.13 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [183]:
from numpy.testing import assert_allclose


res1 = X @ W_diag
res1.sort_indices()

res2 = safe_multiply_broadcast(X, w[None, :])

assert res1.format == res2.format
assert res1.shape == res2.shape
assert_allclose(res1.data, res2.data)